# Scenario Generation in EPyT-Flow

An essential aspect of scenarios are events such as leakages, sensor faults, cyber-attacks, contaminations, actuator events, etc.
Those events create (complex) dynamics in the system directly affecting the operation of the entire WDN.

EPyT-Flow comes with a wide variety of different pre-implemented [events](https://epyt-flow.readthedocs.io/en/stable/tut.events.html) that can be used to build custom scenarios:

- [Leakages](https://epyt-flow.readthedocs.io/en/stable/tut.events.html#leakages) such as abrupt and incipient leakages.
- [Sensor Faults](https://epyt-flow.readthedocs.io/en/stable/tut.events.html#sensor-faults) such as shifts and complete failures.
- [Sensor Reading Attacks](https://epyt-flow.readthedocs.io/en/stable/tut.events.html#sensor-reading-attacks) such as replay and override attacks.
- [Actuator Events](https://epyt-flow.readthedocs.io/en/stable/tut.events.html#actuator-events) such as pump state/speed and valve state events.

Furthermore, EPyT-Flow also allows the user to easily customize those events or implement their own events -- more information can be found in the [documentation](https://epyt-flow.readthedocs.io/en/stable/tut.events.html#custom-events).

#### Outline

This notebook demonstrates how to use EPyT-Flow for creating a scenario that contains multiple different events:

1. Adding leakages
2. Adding sensor faults
    - Before the simulation is run
    - After the simulation as some form of post-processing
3. Exporting simulation results to other data forms (e.g. Excel)
4. Parallel scenario simulation to speed the data generation up

In [ ]:
%pip install epyt-flow

In [ ]:
from epyt.epanet import ToolkitConstants
from epyt_flow.simulation import ScenarioSimulator, ParallelScenarioSimulation, ScenarioConfig
from epyt_flow.simulation.events import AbruptLeakage, SensorFaultStuckZero, SensorFaultGaussian, \
    SENSOR_TYPE_NODE_PRESSURE
from epyt_flow.simulation.scada import ScadaData, ScadaDataXlsxExport
from epyt_flow.data.benchmarks import load_leakdb_scenarios
from epyt_flow.data.networks import load_hanoi
from epyt_flow.utils import to_seconds, plot_timeseries_data

We create a new scenario based on the [Hanoi network](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.data.html#epyt_flow.data.networks.load_hanoi) including a default sensor placement -- note that we also set the flow units to *cubic meter per hour*:

In [ ]:
# Load Hanoi network configuration and make sure that we use "cubic meter per hour" as the flow unit
hanoi_network_config = load_hanoi(include_default_sensor_placement=True, verbose=True,
                                  flow_units_id=ToolkitConstants.EN_CMH)

print(hanoi_network_config.sensor_config.to_json()) # Show sensor config

# Create new scenario simulator
sim = ScenarioSimulator(scenario_config=hanoi_network_config)

# Set simulation duration to 2 days and the hydraulic & reporting time steps to 30min
sim.set_general_parameters(simulation_duration=to_seconds(days=2),
                           hydraulic_time_step=to_seconds(minutes=30),
                           reporting_time_step=to_seconds(minutes=30))

### 1. Adding Leakages

Add an [abrupt leakage](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.events.html#epyt_flow.simulation.events.leakages.AbruptLeakage) at link/pipe "14" -- the leakage is active for 10 hours and starts at 10 hours after simulation begin -- recall that the time arguments are seconds!

In [ ]:
leak = AbruptLeakage(link_id="14", diameter=0.2,
                     start_time=to_seconds(hours=10),
                     end_time=to_seconds(hours=20))

sim.add_leakage(leak)

### 2. Adding Sensor Faults

Add a pressure [sensor fault](https://epyt-flow.readthedocs.io/en/stable/tut.events.html#sensor-faults) (i.e. power failure, sensor readings are set to zero) at node "16" that is active for 6 hours (i.e. starts at 1 day after simulation begin and ends at 30 hours after simulation start).

In [ ]:
sensor_fault = SensorFaultStuckZero(sensor_id="16",
                                    sensor_type=SENSOR_TYPE_NODE_PRESSURE,
                                    start_time=to_seconds(hours=24),
                                    end_time=to_seconds(hours=30))

sim.add_sensor_fault(sensor_fault)

### 3. Exporting Simulation Results

Run the complete simulation to retrieve the SCADA data:

In [ ]:
scada_data = sim.run_simulation()

Retrieve final pressure readings at node "16" as a NumPy array:

In [ ]:
pressure_at_node16 = scada_data.get_data_pressures(sensor_locations=["16"])

In [ ]:
# Plot the pressure readings over time
plot_timeseries_data(pressure_at_node16.T,
                     x_axis_label="Time (30min steps)", y_axis_label="Pressure in $m$")

Note the pressure drop in the beginning (caused by the leakage) and the sensor readings dropping to zero while the sensor fault is active!

Besides retrieving the final sensor readings as NumPy arrays, EPyT-Flow also supports to export the SCADA data to different file formats:
- [EPyT-Flow file format](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.serialization.Serializable.save_to_file)
- [Matlab](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.scada_data_export.ScadaDataMatlabExport)
- [Microsoft Excel](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.scada_data_export.ScadaDataXlsxExport)
- [NumPy](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.scada_data_export.ScadaDataNumpyExport)

Note that only in the first case, the exported data can again be loaded in EPyT-Flow.

Furthermore, note that most EPyT-Flow classes can be exported to a custom file format, and many even to JSON as well -- more details can be found in the [documentation](https://epyt-flow.readthedocs.io/en/stable/tut.serialization.html).

Exporting the complete sensor readings to `my_data_export.xlsx` ([Microsoft Excel export](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.scada_data_export.ScadaDataXlsxExport)):

In [ ]:
ScadaDataXlsxExport(f_out="my_data_export.xlsx").export(scada_data)

In addition to the SCADA data, EPyT-Flow also allows to export the scenario configuration such that it can be restored later or shared with other users.

The complete configuration of the current scenario can be obtained by calling the   [```get_scenario_config()```](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.get_scenario_config) function which returns a [```ScenarioConfig```](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_config.ScenarioConfig) instance:

In [ ]:
my_scenario_config = sim.get_scenario_config()
my_scenario_config

The obtained [```ScenarioConfig```](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_config.ScenarioConfig) instance can be stored and loaded by calling [```save_to_file()```](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.serialization.Serializable.save_to_file) and [```load_from_file()```](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.serialization.Serializable.load_from_file):

In [ ]:
# Save scenario configuration such that it can be shared with other users
my_scenario_config.save_to_file("my_scenario.epytflow_scenario_config")

# Load scenario configuration from file
restored_scenario_config = ScenarioConfig.load_from_file("my_scenario.epytflow_scenario_config")
print(my_scenario_config == restored_scenario_config)  # Check if scenario configs are the same!

#### Post-processing of SCADA data

In [ ]:
# Save SCADA data in a custom file format that allows us to load it back into EPyT-Flow
scada_data.save_to_file("my_data.epytflow_scada_data")

# Load SCADA data back into EPyT-Flow
scada_data = ScadaData.load_from_file("my_data.epytflow_scada_data")

Among some properties, we can change the sensor faults of [`ScadaData`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.scada_data.ScadaData) instance by calling the [`change_sensor_faults()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.scada_data.ScadaData.change_sensor_faults) function.
Note that [`change_sensor_faults()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.scada_data.ScadaData.change_sensor_faults) removes all existing sensor faults before adding the given sensor faults.

Here we change the previously created pressure sensor fault to a different type of sensor fault and also change the time at which the sensor fault is active:

In [ ]:
# Sets a single sensor fault: Adds Gaussian noise to the pressure reading at node "16"
# Note that this overrides all existing sensor faults!
sensor_fault = SensorFaultGaussian(std=.2, sensor_id="16",
                                   sensor_type=SENSOR_TYPE_NODE_PRESSURE,
                                   start_time=to_seconds(hours=31),
                                   end_time=to_seconds(hours=40))
scada_data.change_sensor_faults([sensor_fault])

In [ ]:
# Plot the pressure readings over time
pressure_at_node16 = scada_data.get_data_pressures(sensor_locations=["16"])
plot_timeseries_data(pressure_at_node16.T,
                     x_axis_label="Time (30min steps)", y_axis_label="Pressure in $m$")

Note how the pressure readings changed compared to the previous sensor faults!

#### Close the Simulation

Do not forget to close the simulation by calling the [`close()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.close) function.

In [ ]:
sim.close()

#### 4. Parallel Scenario Simulation

In practice, one might have more than one scenario configuration that have to be simulated. In this context, it would beneficial to use multiple CPU cores to speed the simulation up.

EPyT-Flow supports the simulation of multiple scenarios in parallel -- see [`ParallelScenarioSimulation`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.parallel_simulation.ParallelScenarioSimulation) for details.

For an illustrative example, we load the first 10 [LeakDB](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.data.benchmarks.html#module-epyt_flow.data.benchmarks.leakdb) Net1 scenarios:

In [ ]:
# Load first 10 LeakDB Net1 scenarios
scenarios = load_leakdb_scenarios(range(10), use_net1=True)

Wen simulate those 10 scenarios in parallel using up to 4 CPU scores by utilizing the [`ParallelScenarioSimulation`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.parallel_simulation.ParallelScenarioSimulation) class.
The simulations results ([`ScadaData`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.scada_data.ScadaData) instances) are stored in separate files that can be reloaded into EPyT-Flow:

In [ ]:
def __my_callback(scada_data: ScadaData, _, scenario_idx: int) -> None:
    scada_data.save_to_file(f"Net1_Scenario-ID={scenario_idx}.epytflow_scada_data")

    # Alternatively, it is also possible to return the results:
    #return scada_data

# Run simulations in parallel using up to 4 CPU cores -- export simulation results to ".epytflow_scada_data" files
ParallelScenarioSimulation.run(scenarios, n_jobs=4, callback=__my_callback)  # Returns the results returned in the callback